In [1]:
from ray.tune import Analysis, ExperimentAnalysis

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rc_file
import pandas as pd
from matplotlib import rc_file
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [3]:
import sys
sys.path.append("/p/project/raise-ctp2/cern/particleflow/mlpf")
from utils import count_skipped_configurations

In [4]:
ray_results_folder = "/p/project/raise-ctp2/cern/ray_results/"  # Main folder containing all ray experiments

# Which experiment to analyze
exp_dir = ray_results_folder + "delphes_svr_dataset"

In [5]:
expanalysis = ExperimentAnalysis(exp_dir, default_metric="val_loss", default_mode="min")

In [6]:
result_df = expanalysis.dataframe()

Couldn't read config from 1 paths


In [7]:
hp_df = pd.concat([result_df["trial_id"], result_df.filter(regex="config/*")], axis=1)
hp_df

,trial_id,config/bin_size,config/dropout,config/lr,config/num_graph_layers_id,config/num_graph_layers_reg,config/output_dim,config/weight_decay
0,8275d_00122,64,0.449385,0.009435,0.0,3.0,256,0.000044
1,8275d_00157,64,0.089148,0.000039,4.0,2.0,32,0.001984
2,8275d_00243,16,0.486632,0.000276,2.0,4.0,64,0.000065
3,8275d_00246,32,0.172208,0.000042,3.0,2.0,64,0.000007
4,8275d_00029,32,0.172208,0.000042,3.0,2.0,64,0.000007
...,...,...,...,...,...,...,...,...
294,8275d_00272,8,0.476098,0.000350,1.0,3.0,256,0.012403
295,8275d_00139,64,0.294393,0.004288,3.0,3.0,32,0.011793
296,8275d_00055,64,0.302994,0.000250,2.0,0.0,32,0.005854
297,8275d_00151,8,0.358993,0.000076,1.0,1.0,16,0.000319


In [8]:
learning_curves = {}
for logdir, trial_df in tqdm(expanalysis.trial_dataframes.items(), total=len(expanalysis.trial_dataframes)):
    trial_id = trial_df["trial_id"].values[0]
    learning_curves.update({trial_id: [trial_df, hp_df[hp_df["trial_id"] == trial_id]]})

100%|██████████| 299/299 [00:00<00:00, 3416.71it/s]


In [9]:
list(learning_curves.keys())[:6]

['8275d_00122',
 '8275d_00157',
 '8275d_00243',
 '8275d_00246',
 '8275d_00029',
 '8275d_00014']

In [10]:
example_trial = learning_curves['8275d_00122']
type(example_trial), len(example_trial)

(list, 2)

In [11]:
print('Trial shape:', example_trial[0].shape)
example_trial[0].head(3)

Trial shape: (100, 37)


,charge_loss,cls_acc_unweighted,cls_loss,cos_phi_loss,energy_loss,eta_loss,loss,pt_loss,sin_phi_loss,val_charge_loss,...,date,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time
0,0.116401,0.952096,0.149628,0.003114,4.595759,0.004661,484.088806,0.231505,0.003187,0.042193,...,2022-07-24_20-21-27,1658686887,199.515550,21505,jrc0298,10.14.2.44,199.515550,0,1,0.003106
1,0.048667,0.957321,0.123374,0.001353,4.415458,0.001888,459.301514,0.171240,0.001353,0.036692,...,2022-07-24_20-23-23,1658687003,316.176005,21505,jrc0298,10.14.2.44,316.176005,0,2,0.003106
2,0.043926,0.957641,0.121581,0.001353,4.379020,0.001888,455.343811,0.168168,0.001353,0.034201,...,2022-07-24_20-25-19,1658687119,431.299931,21505,jrc0298,10.14.2.44,431.299931,0,3,0.003106


In [12]:
example_trial[1]

,trial_id,config/bin_size,config/dropout,config/lr,config/num_graph_layers_id,config/num_graph_layers_reg,config/output_dim,config/weight_decay
0,8275d_00122,64,0.449385,0.009435,0.0,3.0,256,0.000044


In [13]:
import pickle
with open('LC_data.pkl', 'wb') as f:
    pickle.dump(learning_curves, f)